### Setup Test Database

In [ ]:
#| hide

from fastsql import *
from sqlalchemy import text
import os
import re
from typing import List, Dict, Any, Optional
from contextlib import contextmanager
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables - use absolute path
notebook_dir = Path().absolute()
if notebook_dir.name == 'nbs':
    env_path = notebook_dir / '.env'
else:
    env_path = notebook_dir / 'nbs' / '.env'

print(f"📁 Loading .env from: {env_path}")
print(f"📄 File exists: {env_path.exists()}")

if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded environment variables")
    print(f"🔑 DB_PASS is {'SET' if os.getenv('DB_PASS') else 'NOT SET'}")
else:
    print(f"⚠️ .env file not found at {env_path}")

# Setup test database using tenant infrastructure (PostgreSQL or SQLite based on env)
from fh_saas.utils_sql import *
from fh_saas.db_tenant import get_or_create_tenant_db
from fh_saas.db_host import timestamp, gen_id

📁 Loading .env from: c:\Users\abhis\Documents\fh-suite\fh-saas\nbs\.env
📄 File exists: True
✅ Loaded environment variables
🔑 DB_PASS is SET


In [ ]:
#| hide

# Create a shared tenant for utilities testing
print("🔧 Setting up test database...")
test_db = get_or_create_tenant_db("test_utils", "Utilities Test Tenant")

# Create transaction table for testing
create_table_sql = """
CREATE TABLE IF NOT EXISTS transaction (
    transaction_id INTEGER PRIMARY KEY,
    transaction_account_id TEXT NOT NULL,
    transaction_account_connection_id TEXT,
    transaction_date TEXT NOT NULL,
    transaction_amount REAL NOT NULL,
    transaction_category TEXT
)
"""
test_db.conn.execute(text(create_table_sql))
test_db.conn.commit()

print(f"✅ Test database ready: {get_db_type()}")
print(f"   Tenant: test_utils")

🔧 Setting up test database...
ℹ️  Tenant exists: Utilities Test Tenant
✅ Test database ready: POSTGRESQL
   Tenant: test_utils


### Test Insert-Only Operations

In [ ]:
#| hide

print("🧪 Testing insert_only...")

# Clean up any existing test data first
test_db.conn.execute(text("DELETE FROM transaction"))
test_db.conn.commit()
print("   🧹 Cleaned up existing test data")

# Test 1: Insert new record
record1 = {
    "transaction_id": 1,
    "transaction_account_id": "acc_001",
    "transaction_account_connection_id": "conn_001",
    "transaction_date": "2024-01-01",
    "transaction_amount": 100.50,
    "transaction_category": "groceries"
}
insert_only(test_db, "transaction", record1, ["transaction_id"])

# Verify inserted
result = get_by_id(test_db, "transaction", 1, "transaction_id")
rows = result.fetchall()
assert len(rows) == 1, "Record should be inserted"
print("   ✅ Single insert successful")

# Test 2: Try inserting duplicate (should be ignored)
record1_dup = {**record1, "transaction_amount": 999.99}  # Different amount
insert_only(test_db, "transaction", record1_dup, ["transaction_id"])

# Verify amount is still original (not updated)
result = get_by_id(test_db, "transaction", 1, "transaction_id")
row = result.fetchone()
assert row[4] == 100.50, f"Amount should not change (duplicate ignored), but got {row[4]}"
print("   ✅ Duplicate insert correctly ignored")

# Test 3: Bulk insert with some duplicates
bulk_records = [
    {"transaction_id": 1, "transaction_account_id": "acc_001", "transaction_date": "2024-01-01", "transaction_amount": 888.88, "transaction_category": "test"},
    {"transaction_id": 2, "transaction_account_id": "acc_002", "transaction_date": "2024-01-02", "transaction_amount": 200.00, "transaction_category": "dining"},
    {"transaction_id": 3, "transaction_account_id": "acc_003", "transaction_date": "2024-01-03", "transaction_amount": 300.00, "transaction_category": "shopping"},
]
bulk_insert_only(test_db, "transaction", bulk_records, ["transaction_id"])

# Verify only 2 new records added (ID 1 already existed)
result = test_db.conn.execute(text("SELECT COUNT(*) FROM transaction"))
count = result.fetchone()[0]
assert count == 3, f"Should have 3 records total, got {count}"
print(f"   ✅ Bulk insert successful: {count} total records")

print("\n✅ Insert-only operations working correctly!")


🧪 Testing insert_only...
   🧹 Cleaned up existing test data
   ✅ Single insert successful
   ✅ Duplicate insert correctly ignored
   ✅ Bulk insert successful: 3 total records

✅ Insert-only operations working correctly!


### Test Upsert Operations

In [ ]:
#| hide

print("🧪 Testing upsert...")

# Test 1: Upsert existing record (should update)
upsert_record = {
    "transaction_id": 1,
    "transaction_account_id": "acc_001_updated",
    "transaction_account_connection_id": "conn_001_updated",
    "transaction_date": "2024-01-01",
    "transaction_amount": 150.75,
    "transaction_category": "groceries_updated"
}
upsert(test_db, "transaction", upsert_record, ["transaction_id"])

# Verify updated
result = get_by_id(test_db, "transaction", 1, "transaction_id")
row = result.fetchone()
assert row[4] == 150.75, f"Amount should be updated to 150.75, got {row[4]}"
assert row[5] == "groceries_updated", "Category should be updated"
print("   ✅ Upsert updated existing record")

# Test 2: Upsert new record (should insert)
new_record = {
    "transaction_id": 10,
    "transaction_account_id": "acc_010",
    "transaction_account_connection_id": "conn_010",
    "transaction_date": "2024-01-10",
    "transaction_amount": 1000.00,
    "transaction_category": "salary"
}
upsert(test_db, "transaction", new_record, ["transaction_id"])

# Verify inserted
result = get_by_id(test_db, "transaction", 10, "transaction_id")
rows = result.fetchall()
assert len(rows) == 1, "New record should be inserted"
print("   ✅ Upsert inserted new record")

# Test 3: Bulk upsert (mix of new and existing)
bulk_upsert_records = [
    {"transaction_id": 1, "transaction_account_id": "acc_001", "transaction_date": "2024-01-01", "transaction_amount": 175.00, "transaction_category": "groceries_final"},
    {"transaction_id": 2, "transaction_account_id": "acc_002", "transaction_date": "2024-01-02", "transaction_amount": 250.00, "transaction_category": "dining_updated"},
    {"transaction_id": 20, "transaction_account_id": "acc_020", "transaction_date": "2024-01-20", "transaction_amount": 500.00, "transaction_category": "rent"},
    {"transaction_id": 21, "transaction_account_id": "acc_021", "transaction_date": "2024-01-21", "transaction_amount": 600.00, "transaction_category": "utilities"},
]
bulk_upsert(test_db, "transaction", bulk_upsert_records, ["transaction_id"])

# Verify results
result = get_by_id(test_db, "transaction", 1, "transaction_id")
row = result.fetchone()
assert row[4] == 175.00, "ID 1 should be updated to 175.00"

result = test_db.conn.execute(text("SELECT COUNT(*) FROM transaction"))
count = result.fetchone()[0]
# Count: 3 original (1,2,3) + 1 new upsert (10) + 2 new bulk (20,21) = 6 total
# IDs 1 and 2 from bulk were updates, not new inserts
assert count == 6, f"Should have 6 records total (IDs: 1,2,3,10,20,21), got {count}"
print(f"   ✅ Bulk upsert successful: {count} total records")

print("\n✅ Upsert operations working correctly!")

🧪 Testing upsert...
   ✅ Upsert updated existing record
   ✅ Upsert inserted new record
   ✅ Bulk upsert successful: 6 total records

✅ Upsert operations working correctly!


### Test CRUD Operations

In [ ]:
#| hide

print("🧪 Testing CRUD operations...")

# Test get_by_id
result = get_by_id(test_db, "transaction", 1, "transaction_id")
row = result.fetchone()
assert row is not None, "Should retrieve record with ID 1"
print("   ✅ get_by_id works")

# Test update_record
update_record(test_db, "transaction", 1, "transaction_id", 
              transaction_amount=999.99, transaction_category="updated_category")
result = get_by_id(test_db, "transaction", 1, "transaction_id")
row = result.fetchone()
assert row[4] == 999.99, "Amount should be updated"
assert row[5] == "updated_category", "Category should be updated"
print("   ✅ update_record works")

# Test delete_record
delete_record(test_db, "transaction", 20, "transaction_id")
result = get_by_id(test_db, "transaction", 20, "transaction_id")
rows = result.fetchall()
assert len(rows) == 0, "Record should be deleted"
print("   ✅ delete_record works")

# Test bulk_delete
bulk_delete(test_db, "transaction", [10, 21], "transaction_id")
result = test_db.conn.execute(text("SELECT COUNT(*) FROM transaction WHERE transaction_id IN (10, 21)"))
count = result.fetchone()[0]
assert count == 0, "Both records should be deleted"
print("   ✅ bulk_delete works")

result = test_db.conn.execute(text("SELECT COUNT(*) FROM transaction"))
remaining = result.fetchone()[0]
print(f"   📊 Remaining records: {remaining}")

print("\n✅ CRUD operations working correctly!")

🧪 Testing CRUD operations...
   ✅ get_by_id works
   ✅ update_record works
   ✅ delete_record works
   ✅ bulk_delete works
   📊 Remaining records: 3

✅ CRUD operations working correctly!


### Test Query Registry & Executor

In [ ]:
#| hide

# Example SQL Query Registry (not exported - provide your own in your app)
# This is just for demonstration and testing purposes

SQL_REGISTRY = {
    # Transaction Queries
    "transaction_by_id": """
        SELECT * FROM transaction 
        WHERE transaction_id = :transaction_id
    """,
    
    "transaction_by_connection_id": """
        SELECT * FROM transaction 
        WHERE transaction_account_connection_id = :connection_id 
        ORDER BY transaction_date DESC
    """,
    
    "transaction_by_account_id": """
        SELECT * FROM transaction 
        WHERE transaction_account_id = :account_id 
        ORDER BY transaction_date DESC
    """,
    
    "transaction_by_date_range": """
        SELECT * FROM transaction 
        WHERE transaction_date >= :start_date 
        AND transaction_date <= :end_date 
        ORDER BY transaction_date DESC
    """,
}

# Query ID Constants (define these in your app for type safety)
QRY_TRANSACTION_BY_ID = "transaction_by_id"
QRY_TRANSACTION_BY_CONNECTION = "transaction_by_connection_id"
QRY_TRANSACTION_BY_ACCOUNT = "transaction_by_account_id"
QRY_TRANSACTION_BY_DATE_RANGE = "transaction_by_date_range"

In [ ]:
#| hide

print("🧪 Testing query registry and run_id...")

# Test query by ID
result = run_id(test_db, SQL_REGISTRY, QRY_TRANSACTION_BY_ID, {"transaction_id": 1})
rows = result.fetchall()
assert len(rows) == 1, "Should find transaction with ID 1"
print(f"   ✅ {QRY_TRANSACTION_BY_ID} works")

# Test query by account ID
result = run_id(test_db, SQL_REGISTRY, QRY_TRANSACTION_BY_ACCOUNT, {"account_id": "acc_002"})
rows = result.fetchall()
assert len(rows) >= 1, "Should find transactions for account acc_002"
print(f"   ✅ {QRY_TRANSACTION_BY_ACCOUNT} works")

# Test query by date range
result = run_id(test_db, SQL_REGISTRY, QRY_TRANSACTION_BY_DATE_RANGE, {
    "start_date": "2024-01-01",
    "end_date": "2024-01-31"
})
rows = result.fetchall()
assert len(rows) >= 1, "Should find transactions in January 2024"
print(f"   ✅ {QRY_TRANSACTION_BY_DATE_RANGE} works ({len(rows)} results)")

# Test missing parameters (should raise error)
try:
    run_id(test_db, SQL_REGISTRY, QRY_TRANSACTION_BY_ID, {})  # Missing transaction_id
    assert False, "Should have raised ValueError for missing param"
except ValueError as e:
    assert "Missing required parameters" in str(e)
    print("   ✅ Parameter validation works")

# Test invalid query ID (should raise error)
try:
    run_id(test_db, SQL_REGISTRY, "nonexistent_query", {})
    assert False, "Should have raised ValueError for invalid query ID"
except ValueError as e:
    assert "not found in registry" in str(e)
    print("   ✅ Query ID validation works")

print("\n✅ Query registry and executor working correctly!")


🧪 Testing query registry and run_id...
   ✅ transaction_by_id works
   ✅ transaction_by_account_id works
   ✅ transaction_by_date_range works (3 results)
   ✅ Parameter validation works
   ✅ Query ID validation works

✅ Query registry and executor working correctly!


### Test Helper Utilities

In [ ]:
#| hide

print("🧪 Testing helper utilities...")

# Test pagination
base_query = "SELECT * FROM transaction ORDER BY transaction_id"
paginated = paginate_sql(base_query, page=2, page_size=2)
assert "LIMIT 2 OFFSET 2" in paginated, "Pagination should add LIMIT and OFFSET"
print("   ✅ paginate_sql works")

# Execute paginated query
result = test_db.conn.execute(text(paginated))
rows = result.fetchall()
print(f"   📊 Page 2 results: {len(rows)} records")

# Test transaction context manager - SUCCESS CASE
print("   Testing with_transaction (success case)...")
try:
    with with_transaction(test_db):
        insert_only(test_db, "transaction", {
            "transaction_id": 100,
            "transaction_account_id": "acc_100",
            "transaction_date": "2024-02-01",
            "transaction_amount": 100.00,
            "transaction_category": "test"
        }, ["transaction_id"], auto_commit=False)  # ← auto_commit=False
        # Should commit automatically by context manager
    
    result = get_by_id(test_db, "transaction", 100, "transaction_id")
    rows = result.fetchall()
    assert len(rows) == 1, "Transaction should commit successfully"
    print("   ✅ with_transaction commits on success")
except Exception as e:
    print(f"   ❌ Transaction test failed: {e}")

# Test transaction rollback on error - ROLLBACK CASE
print("   Testing with_transaction (rollback case)...")
initial_count_result = test_db.conn.execute(text("SELECT COUNT(*) FROM transaction"))
initial_count = initial_count_result.fetchone()[0]

try:
    with with_transaction(test_db):
        insert_only(test_db, "transaction", {
            "transaction_id": 200,
            "transaction_account_id": "acc_200",
            "transaction_date": "2024-03-01",
            "transaction_amount": 200.00,
            "transaction_category": "test"
        }, ["transaction_id"], auto_commit=False)  # ← auto_commit=False for rollback to work
        raise Exception("Simulated error")  # Force rollback
except Exception:
    pass  # Expected

final_count_result = test_db.conn.execute(text("SELECT COUNT(*) FROM transaction"))
final_count = final_count_result.fetchone()[0]
assert final_count == initial_count, f"Transaction should rollback on error (expected {initial_count}, got {final_count})"
print("   ✅ with_transaction rolls back on error")

print("\n✅ Helper utilities working correctly!")


🧪 Testing helper utilities...
   ✅ paginate_sql works
   📊 Page 2 results: 1 records
   Testing with_transaction (success case)...
   ✅ with_transaction commits on success
   Testing with_transaction (rollback case)...
   ✅ with_transaction rolls back on error

✅ Helper utilities working correctly!


### Test Summary

In [ ]:
#| hide

print("\n" + "="*60)
print("🎉 ALL SQL UTILITIES TESTS PASSED!")
print("="*60)

# Final statistics
result = test_db.conn.execute(text("SELECT COUNT(*) FROM transaction"))
total_records = result.fetchone()[0]

print(f"\n📊 Test Statistics:")
print(f"   Total records in test database: {total_records}")
print(f"   Query registry size: {len(SQL_REGISTRY)} queries")
print(f"   Database type: {get_db_type()}")
print(f"   Money helpers: to_cents(), from_cents()")

print("\n✅ SQL Utilities Library Ready for Production!")


🎉 ALL SQL UTILITIES TESTS PASSED!

📊 Test Statistics:
   Total records in test database: 4
   Query registry size: 4 queries
   Database type: POSTGRESQL
   Money helpers: to_cents(), from_cents()

✅ SQL Utilities Library Ready for Production!


### Test Money Conversion Helpers

In [ ]:
#| hide

print("🧪 Testing money conversion helpers...")

# Test to_cents - basic conversions
assert to_cents("150.00") == 15000, "String dollars to cents"
assert to_cents("0.99") == 99, "Sub-dollar amount"
assert to_cents("-25.50") == -2550, "Negative amount"
assert to_cents(150.0) == 15000, "Float dollars to cents"
assert to_cents(0.01) == 1, "One cent"
print("   ✅ to_cents basic conversions work")

# Test to_cents - edge cases
assert to_cents(None) is None, "None input returns None"
assert to_cents('') is None, "Empty string returns None"
assert to_cents("invalid") is None, "Invalid string returns None"
assert to_cents(0) == 0, "Zero returns 0"
assert to_cents("0.00") == 0, "Zero string returns 0"
print("   ✅ to_cents edge cases work")

# Test from_cents - basic conversions
assert from_cents(15000) == "$150.00", "Cents to formatted dollars"
assert from_cents(99) == "$0.99", "Sub-dollar cents"
assert from_cents(-2550) == "-$25.50", "Negative cents"
assert from_cents(1) == "$0.01", "One cent"
assert from_cents(100000) == "$1,000.00", "Large amount with comma"
print("   ✅ from_cents basic conversions work")

# Test from_cents - edge cases
assert from_cents(None) == "$0.00", "None returns $0.00"
assert from_cents(0) == "$0.00", "Zero returns $0.00"
print("   ✅ from_cents edge cases work")

# Test round-trip conversion
original = "123.45"
cents = to_cents(original)
assert cents == 12345, "Convert to cents"
# Note: from_cents returns formatted string with $, so we strip it for comparison
formatted = from_cents(cents)
assert formatted == "$123.45", "Convert back to dollars"
print("   ✅ Round-trip conversion works")

print("\n✅ Money conversion helpers working correctly!")

🧪 Testing money conversion helpers...
   ✅ to_cents basic conversions work
   ✅ to_cents edge cases work
   ✅ from_cents basic conversions work
   ✅ from_cents edge cases work
   ✅ Round-trip conversion works

✅ Money conversion helpers working correctly!


: 